In [ ]:
import requests

# API 엔드포인트
url = "http://apis.data.go.kr/1613000/RTMSDataSvcAptTrade/getRTMSDataSvcAptTrade"

service_key = "jYmQgKsPeM/KffLGjHFgvyTBGB/6ODgIGe2qZjXfsd6NG9d4jMtfl7gmx1MUZHpc65LVhAG3ChnTrqHlBr3AHQ=="
gu_code = "11215"  # 서울 광진구
base_date = "202401"  # 조회 연월 (YYYYMM)

# API 요청 파라미터
params = {
    "LAWD_CD": gu_code,
    "DEAL_YMD": base_date,
    "serviceKey": service_key  # Decoding된 인증키 사용
}

# API 호출
res = requests.get(url, params=params)

# 응답 코드 및 내용 확인
print(f"응답 코드: {res.status_code}")
print(res.text)

응답 코드: 200
<?xml version="1.0" encoding="utf-8" standalone="yes"?><response><header><resultCode>000</resultCode><resultMsg>OK</resultMsg></header><body><items><item><aptDong> </aptDong><aptNm>아차산인앤인W클래스101동</aptNm><buildYear>2007</buildYear><buyerGbn>개인</buyerGbn><cdealDay> </cdealDay><cdealType> </cdealType><dealAmount>87,000</dealAmount><dealDay>16</dealDay><dealMonth>1</dealMonth><dealYear>2024</dealYear><dealingGbn>중개거래</dealingGbn><estateAgentSggNm>서울 광진구</estateAgentSggNm><excluUseAr>116.36</excluUseAr><floor>7</floor><jibun>58-26</jibun><landLeaseholdGbn>N</landLeaseholdGbn><rgstDate>24.05.31</rgstDate><sggCd>11215</sggCd><slerGbn>개인</slerGbn><umdNm>구의동</umdNm></item><item><aptDong>A</aptDong><aptNm>더샵스타시티</aptNm><buildYear>2007</buildYear><buyerGbn>개인</buyerGbn><cdealDay> </cdealDay><cdealType> </cdealType><dealAmount>110,000</dealAmount><dealDay>31</dealDay><dealMonth>1</dealMonth><dealYear>2024</dealYear><dealingGbn>직거래</dealingGbn><estateAgentSggNm> </estateAgentSggNm><exclu

In [11]:
import pandas as pd
import xml.etree.ElementTree as ET

def get_items(response):
    root = ET.fromstring(response.content)
    item_list = []
    for child in root.find('body').find('items'):
        elements = child.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip()
            # print tag, text
            data[tag] = text
        item_list.append(data)  
    return item_list
    
items_list = get_items(res)
items = pd.DataFrame(items_list) 
items.head()

,aptDong,aptNm,buildYear,buyerGbn,cdealDay,cdealType,dealAmount,dealDay,dealMonth,dealYear,dealingGbn,estateAgentSggNm,excluUseAr,floor,jibun,landLeaseholdGbn,rgstDate,sggCd,slerGbn,umdNm
0,,아차산인앤인W클래스101동,2007,개인,,,"87,000",16,1,2024,중개거래,서울 광진구,116.36,7,58-26,N,24.05.31,11215,개인,구의동
1,A,더샵스타시티,2007,개인,,,"110,000",31,1,2024,직거래,,96.98,18,227-7,N,24.02.29,11215,개인,자양동
2,5,극동1,1985,개인,,,"220,000",30,1,2024,중개거래,서울 광진구,156.92,8,218-1,N,24.03.15,11215,개인,광장동
3,1403,광장힐스테이트,2012,개인,,,"211,000",26,1,2024,중개거래,서울 광진구,130.17,19,582,N,24.04.05,11215,개인,광장동
4,210,구의현대2단지,1996,개인,,,"142,000",31,1,2024,중개거래,서울 광진구,84.86,25,611,N,24.04.30,11215,개인,구의동


In [17]:
code_file = r"C:\Users\jee36\Desktop\데이터_project\아파트실거래가예측\법정동코드 전체자료.txt"
code = pd.read_csv(code_file, sep='\t', encoding='utf-8')

# 데이터 확인
print(code.head())

        법정동코드           법정동명 폐지여부
0  1100000000          서울특별시   존재
1  1111000000      서울특별시 종로구   존재
2  1111010100  서울특별시 종로구 청운동   존재
3  1111010200  서울특별시 종로구 신교동   존재
4  1111010300  서울특별시 종로구 궁정동   존재


In [18]:
code.columns = ['code', 'name', 'is_exist']

In [19]:
code = code [code['is_exist'] == '존재']

In [20]:
print(code['code'][0])
print(type(code['code'][0])) ## int64타입

## string으로 변경
code['code'] = code['code'].apply(str) 

1100000000
<class 'numpy.int64'>


In [21]:
# 2018-2024년도까지의 데이터를 수집하기 위한 연월 리스트 생성
year = [str("%02d" %(y)) for y in range(2018, 2024)]
month = [str("%02d" %(m)) for m in range(1, 13)]
base_date_list = ["%s%s" %(y, m) for y in year for m in month ]

In [26]:
gu = "강남구"
gu_code = code[ (code['name'].str.contains(gu) )]
gu_code = gu_code['code'].reset_index(drop=True)
gu_code = str(gu_code[0])[0:5]
print(gu_code)

11680


In [27]:
import requests
import datetime

def get_data(gu_code, base_date):
    url = "http://apis.data.go.kr/1613000/RTMSDataSvcAptTrade/getRTMSDataSvcAptTrade"
    service_key = "jYmQgKsPeM/KffLGjHFgvyTBGB/6ODgIGe2qZjXfsd6NG9d4jMtfl7gmx1MUZHpc65LVhAG3ChnTrqHlBr3AHQ=="
    params = {
    "LAWD_CD": gu_code,
    "DEAL_YMD": base_date,
    "serviceKey": service_key  # Decoding된 인증키 사용
    }

    res = requests.get(url, params=params)
    return res

In [28]:
items_list = []
for base_date in base_date_list:
    res = get_data(gu_code, base_date)
    items_list += get_items(res)
    
len(items_list)

720

In [30]:
import pandas as pd
import os

# items_list의 길이 확인
print("Length of items_list:", len(items_list))

# items_list의 일부 내용 확인
print("First 5 items in items_list:", items_list[:5])

# 데이터프레임 생성
items = pd.DataFrame(items_list)
print("DataFrame shape:", items.shape)
print("First 5 rows of DataFrame:")
print(items.head())

# 저장할 경로와 파일 이름 지정
save_path = r"C:\Users\jee36\Desktop\데이터_project\아파트실거래가예측\국토부실거래가_API"
file_name = "%s_%s~%s.csv" % (gu, year[0], year[-1])
full_path = os.path.join(save_path, file_name)

# 데이터프레임을 CSV 파일로 저장
items.to_csv(full_path, index=False, encoding="euc-kr")

# 저장된 파일의 내용 확인
saved_items = pd.read_csv(full_path, encoding="euc-kr")
print("Saved DataFrame shape:", saved_items.shape)
print("First 5 rows of saved DataFrame:")
print(saved_items.head())

Length of items_list: 720
First 5 items in items_list: [{'aptDong': '', 'aptNm': '래미안강남힐즈', 'buildYear': '2014', 'buyerGbn': '', 'cdealDay': '', 'cdealType': '', 'dealAmount': '110,000', 'dealDay': '10', 'dealMonth': '1', 'dealYear': '2018', 'dealingGbn': '', 'estateAgentSggNm': '', 'excluUseAr': '91.96', 'floor': '15', 'jibun': '611', 'landLeaseholdGbn': 'N', 'rgstDate': '', 'sggCd': '11680', 'slerGbn': '', 'umdNm': '자곡동'}, {'aptDong': '', 'aptNm': '롯데캐슬프레미어', 'buildYear': '2007', 'buyerGbn': '', 'cdealDay': '', 'cdealType': '', 'dealAmount': '245,000', 'dealDay': '6', 'dealMonth': '1', 'dealYear': '2018', 'dealingGbn': '', 'estateAgentSggNm': '', 'excluUseAr': '213.435', 'floor': '7', 'jibun': '11', 'landLeaseholdGbn': 'N', 'rgstDate': '', 'sggCd': '11680', 'slerGbn': '', 'umdNm': '삼성동'}, {'aptDong': '', 'aptNm': '성원대치2단지아파트', 'buildYear': '1992', 'buyerGbn': '', 'cdealDay': '', 'cdealType': '', 'dealAmount': '95,000', 'dealDay': '19', 'dealMonth': '1', 'dealYear': '2018', 'dealingGb

In [29]:
import pandas as pd
import os

save_path = r"C:\Users\jee36\Desktop\데이터_project\아파트실거래가예측\국토부실거래가_API"
file_name = "%s_%s~%s.csv" % (gu, year[0], year[-1])
full_path = os.path.join(save_path, file_name)

# 데이터프레임을 CSV 파일로 저장
items.to_csv(full_path, index=False, encoding="euc-kr")